## 7.7 パラメータチューニング

In [ ]:
library(caret)
library(tidyverse)
my_data <- cars
my_model <- train(form = dist ~ speed, data = my_data, method = "knn")
my_model$results
#>   k     RMSE  Rsquared      MAE   RMSESD RsquaredSD    MAESD
#> 1 5 15.72114 0.6615765 12.54588 3.013243 0.11043907 2.372245
#> 2 7 16.19047 0.6601173 12.68464 3.165330 0.09990158 2.329326
#> 3 9 16.30276 0.6556700 12.84811 3.367423 0.09645747 2.471620

In [ ]:
my_params <- expand.grid(k = 1:15)

my_model <- train(form = dist ~ speed, data = my_data, method = "knn",
                  tuneGrid = my_params,
                  trControl = trainControl(method = "LOOCV"))

In [ ]:
head(my_model$results)
#>   k     RMSE  Rsquared      MAE
#> 1 1 17.22299 0.5777197 13.84900
#> 2 2 16.81462 0.5936438 13.03469
#> 3 3 16.32874 0.6218866 12.74524
#> 4 4 15.98970 0.6086993 12.27888
#> 5 5 15.79924 0.6169267 11.96067
#> 6 6 15.98720 0.6079396 12.26667

In [ ]:
ggplot(my_model)

In [ ]:
my_model$bestTune
#>   k
#> 5 5

In [ ]:
my_model$results %>%
  filter(RMSE == min(RMSE))
#>   k     RMSE  Rsquared      MAE
#> 1 5 15.79924 0.6169267 11.96067

In [ ]:
y  <- my_data$dist
y_ <- my_model %>% predict(my_data)
RMSE(y_, y)
#> [1] 13.96845

### 7.7.1 補足：ハイパーパラメータとRMSE（訓練）

In [ ]:
library(caret)
library(tidyverse)
my_data <- cars

my_loocv <- function(k) {
  my_model <- train(form = dist ~ speed, data = my_data, method = "knn",
                    tuneGrid = data.frame(k = k),
                    trControl = trainControl(method = "LOOCV"))
  y  <- my_data$dist
  y_ <- my_model %>% predict(my_data)
  list(k = k,
       training = RMSE(y_, y),             # RMSE（訓練）
       validation = my_model$results$RMSE) # RMSE（検証）
}

my_results <- 1:15 %>% map_dfr(my_loocv)

In [ ]:
my_results %>%
  pivot_longer(-k) %>%
  ggplot(aes(x = k, y = value,
             color = name)) +
  geom_line() + geom_point() +
  xlab("#Neighbors") + ylab("RMSE") +
  theme(legend.position = c(1, 0),
        legend.justification = c(1, 0))